In [4]:
import pandas as pd
import numpy as np
import re
import csv
import codecs

In [ ]:
# Load and format the external datasets
for i in range(1, 7):
    filename = f'books_with_description{i}.csv'
    locals()["df" + str(i)] = pd.read_csv(filename)

# Format for each dataset
df_ext1 = df1[['book_isbn', 'book_desc']]
df_ext1 = df_ext1.rename(columns={'book_isbn': 'isbn'})
df_ext1 = df_ext1.rename(columns={'book_desc': 'description'})
df_ext1['isbn'] = df_ext1['isbn'].astype(str)
df_ext1['isbn'] = df_ext1['isbn'].str.replace('13:', '')
df_ext1['isbn'] = df_ext1['isbn'].apply(lambda x: '{:.0f}'.format(float(x)) if 'E' in x else x.ljust(13, '0'))

df_ext2 = df2[['isbn13', 'description']]
df_ext2 = df_ext2.rename(columns={'isbn13': 'isbn'})
df_ext2['isbn'] = df_ext2['isbn'].astype(str)
df_ext2['isbn'] = df_ext2['isbn'].str.replace(r'\.0$', '', regex=True)

df_ext3 = df3[['isbn13', 'description']]
df_ext3 = df_ext3.rename(columns={'isbn13': 'isbn'})
df_ext3['isbn'] = df_ext3['isbn'].astype(str)
df_ext3['isbn'] = df_ext3['isbn'].str.replace(r'\.0$', '', regex=True)

df_ext4 = df4[['isbn', 'description']]
df_ext4['isbn'] = df_ext4['isbn'].astype(str)
df_ext4 = df_ext4[df_ext4['isbn'].str.contains(r'^[eE]+$')]
df_ext4['isbn'] = df_ext4['isbn'].apply(lambda x: '{:.0f}'.format(float(x)) if 'E' in x else x.ljust(13, '0'))

df_ext5 = df5[['isbn13', 'description']]
df_ext5 = df_ext5.rename(columns={'isbn13': 'isbn'})
df_ext5['isbn'] = df_ext5['isbn'].astype(str)
df_ext5['isbn'] = df_ext5['isbn'].str.replace(r'\.0$', '', regex=True)

df_ext6 = df6[['isbn', 'description']]
df_ext6['isbn'] = df_ext6['isbn'].astype(str)
df_ext6['isbn'] = df_ext6['isbn'].str.replace('13:', '')
df_ext6 = df_ext6[df_ext6['isbn'].str.contains(r'^[eE]+$')]
df_ext6['isbn'] = df_ext6['isbn'].apply(lambda x: '{:.0f}'.format(float(x)) if 'E' in x else x.ljust(13, '0'))

# Join and clean dataframes
combined_df = pd.concat([df_ext1, df_ext2, df_ext3, df_ext4, df_ext5, df_ext6])
combined_df['isbn'] = combined_df['isbn'].replace('nan', np.nan)
combined_df = combined_df.dropna()
combined_df = combined_df.drop_duplicates(subset='isbn')
combined_df['isbn'] = combined_df['isbn'].str.replace('X', '10')

In [48]:
# Fix manually the broken format in the delivered file
with open('books.csv', 'rb') as input_file:

    decoder = codecs.iterdecode(input_file, 'utf-8', errors='replace')
    reader = csv.reader(decoder)
    rows = []
    
    # Modify the broken rows
    for i, row in enumerate(reader):
        if i == 3349:
            row[2] = "Sam Bass Warner Jr./Sam B. Warner"
            row[3] = "3.58"
            row[4] = "0674842111"
            row[5] = "9780674842113"
            row[6] = "en-US"
            row[7] = "236"
            row[8] = "61"
            row[9] = "6"
            row[10] = "4/20/2004"
            row[11] = "Harvard University Press"
            del row[12] 
        if i == 4703:
            row[2] = "David E. Smith"
            row[3] = "3.58"
            row[4] = "1593600119"
            row[5] = "9781593600112"
            row[6] = "eng"
            row[7] = "400"
            row[8] = "26"
            row[9] = "4"
            row[10] = "4/6/2004"
            row[11] = "Cold Spring Press"
            del row[12] 
        if i == 5878:
            row[2] = "James Wesley Rawles"
            row[3] = "3.63"
            row[4] = "156384155X"
            row[5] = "9781563841552"
            row[6] = "eng"
            row[7] = "342"
            row[8] = "38"
            row[9] = "4"
            row[10] = "1/15/1999"
            row[11] = "Huntington House Publishers"
            del row[12] 
        if i == 8980:
            row[2] = "Brown Son & Ferguson"
            row[3] = "0.00"
            row[4] = "0851742718"
            row[5] = "9780851742717"
            row[6] = "eng"
            row[7] = "49"
            row[8] = "0"
            row[9] = "0"
            row[10] = "5/1/1977"
            row[11] = "Brown Son & Ferguson Ltd."
            del row[12] 

        # Append the row to the list
        rows.append(row)

# Open the output CSV file
with open('books_clean.csv', 'w', newline='', encoding='utf-8') as output_file:
    # Create a CSV writer object
    writer = csv.writer(output_file)
    
    # Write the modified rows to the output file
    writer.writerows(rows)

In [112]:
# Load and transform the delivered dataset
df = pd.read_csv('books_clean.csv')
df_imp = df[['isbn13', 'title', 'authors', 'publisher', 'average_rating']]
df_imp = df_imp.rename(columns={'isbn13': 'isbn'})
df_imp = df_imp.rename(columns={'authors': 'writer'})
df_imp = df_imp.rename(columns={'average_rating': 'avg_score'})
df_imp['isbn'] = df_imp['isbn'].astype(str).str.rstrip('.0').str.ljust(13, '0')
df_imp['isbn'] = df_imp['isbn'].str.replace('X', '10')

# Save the transformed set for direct database import
df_imp.to_csv('books_import.csv', index=False)

In [113]:
# Intersection between the delivered and the scraped dataset
merged_df = pd.merge(combined_df, df_imp, on='isbn', how='inner')
book_desc_df = merged_df[['isbn', 'description']]

print("Number of matching books found:", len(book_desc_df), " of ", len(df_imp))

Number of matching books found: 8842  of  11127


In [114]:
# Save the final reduced description dataset
book_desc_df.to_csv('books_descriptions.csv', index=False)

In [115]:
# Display the total number of unique descriptions found
len(combined_df)

1045834